Steps are:

- select a reconstruction time
- the code determines which paleogeography stage this falls within, gets the start and end times
- load the relevant precomputed multipoint files, and in the process assign an integer to the different types for use in interpolation steps (e.g. set land to be 1, shallow marine to be 2, etc)

- for land and marine



In [1]:
import sys
sys.path.append('../paleogeography')
sys.path.append('../PlateTectonicTools/')

import pygplates
import glob, re
import os
import numpy as np
from scipy import interpolate

import polygon_processing as pp
import paleogeography as pg
import paleogeography_tweening as pgt
import paleotopography as pt

from proximity_query import *
from create_gpml import create_gpml_regular_long_lat_mesh
import points_in_polygons
from sphere_tools import sampleOnSphere
import points_spatial_tree

import tempfile

reconstruction_basedir = '../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles/'
tween_basedir = './tween_feature_collections/'
file_format = 'gpmlz'

output_dir = './Paleotopography_Grids/'

netcdf3_output = False

rotation_model = pygplates.RotationModel(['%s/Global_EB_250-0Ma_GK07_Matthews++.rot' % reconstruction_basedir,
                                          '%s/Global_EB_410-250Ma_GK07_Matthews++.rot' % reconstruction_basedir])
rotation_file = '%s/Global_EarthByte_230-0Ma_GK07_AREPS.rot' % reconstruction_basedir
#MODELDIR = '/Users/simon/GIT/agegrid-dev/input_files/Global_Model_WD_Internal_Release_2016_v5/'
#rotation_file = ['%s/Global_EB_250-0Ma_GK07_2016_v5.rot' % MODELDIR,\
#                 '%s/Global_EB_410-250Ma_GK07_2016_v5.rot' % MODELDIR]




COBterrane_file = '%s/Global_EarthByte_GeeK07_COB_Terranes_Matthews_etal.gpml' % reconstruction_basedir

#agegrid_file_template = '/Users/Simon/Data/AgeGrids/Agegrids_30m_20151002_2015_v1_r756/agegrid_30m_%d.grd'
agegrid_file_template = '/Users/Simon/cloudstor/age_grids/2016_v5_r1031/raw/sphtmp_mask_%0.1fMa.nc'


#############################################
## Set the heights for different environment
#############################################
depth_for_unknown_ocean = -1000
# ----------------------------------
shallow_marine_elevation = -200.
# ----------------------------------
lowland_elevation = 200.
# ----------------------------------
max_mountain_elevation = 1500.
# NOTE - this height is actually the mountain height IN ADDITION TO the lowland height
# so that the maximum absolute elevation would be [lowland_elevation + max_mountain_elevation]
# TODO should call this 'mountain_relief'???
#############################################

# the grid sampling for the output
sampling = 1.0

# this number controls how small polygons are exclude when merging the COB terranes into 
# land/sea masking polygons
area_threshold = 0.0001

# used for quadtree
subdivision_depth = 2

# this buffer defines the smoothness of the topography at the transition from 'lowland' to 'mountain'
# the distance defined here is the distance over which heights ramp from the lowland elevation to the 
# mountain elevation defined above. (the ramping takes place from the edge of the mountain range inwards
# towards the mountain interior). Any parts of the mountain range greater than this buffer distance from 
# the edge will have a uniform height equal to max_mountain_elevation
mountain_buffer_distance_degrees = 0.001
#mountain_buffer_distance_degrees = 2.

# choose here either 'ocean' or 'land'
# this determines which grid takes precedence where both the age grid and the 
# paleogeographies overlap and contain valid values
land_or_ocean_precedence = 'land'

# this number is used in the final grdfilter step to smooth the output 
# NOTE this value is ignored if 'merge_with_bathymetry' is set to False
grid_smoothing_wavelength_kms = 400.

time_min = 6.
time_max = 401.
time_step = 5.

merge_with_bathymetry = False


####################################################

# make a sorted list of the (midpoint) times for paleogeography polygons
tmp = glob.glob('%s/*/' % reconstruction_basedir)

paleogeography_timeslice_list = []
for tm in tmp:
    paleogeography_timeslice_list.append(float(re.findall(r'\d+Ma+',tm)[1][:-2]))

paleogeography_timeslice_list.sort()

paleogeography_timeslice_list = np.array(paleogeography_timeslice_list)

print(paleogeography_timeslice_list)


Failed to load plotting dependencies
[  6.  14.  22.  33.  45.  53.  76.  90. 105. 126. 140. 152. 169. 195.
 218. 232. 255. 277. 287. 302. 328. 348. 368. 396.]


In [2]:
from joblib import Parallel, delayed

# prepend 0 to the list, since this is not covered in published paleogeography sequence
paleogeography_timeslice_list = np.hstack((0,paleogeography_timeslice_list))
 
num_cpus = 1

#time_min = 260.
#time_max = 261.
#time_step = 5.

if num_cpus==1:
    
    for reconstruction_time in np.arange(time_min,time_max+time_step,time_step):
        if paleogeography_timeslice_list[-1]<=reconstruction_time:break #reconstruction time out of range
        pt.paleotopography_job(reconstruction_time, paleogeography_timeslice_list, 
                               tween_basedir, reconstruction_basedir, output_dir, 
                               file_format, rotation_file, COBterrane_file, agegrid_file_template,
                               lowland_elevation, shallow_marine_elevation, max_mountain_elevation, depth_for_unknown_ocean, 
                               sampling, mountain_buffer_distance_degrees, area_threshold,
                               grid_smoothing_wavelength_kms, merge_with_bathymetry, land_or_ocean_precedence,
                               netcdf3_output)

else:
    Parallel(n_jobs=num_cpus)(delayed(pt.paleotopography_job) \
                              (reconstruction_time, paleogeography_timeslice_list, 
                               tween_basedir, reconstruction_basedir, output_dir, 
                               file_format, rotation_file, COBterrane_file, agegrid_file_template,
                               lowland_elevation, shallow_marine_elevation, max_mountain_elevation, depth_for_unknown_ocean, 
                               sampling, mountain_buffer_distance_degrees, area_threshold,
                               grid_smoothing_wavelength_kms, merge_with_bathymetry, land_or_ocean_precedence,
                               netcdf3_output)
                              for reconstruction_time in np.arange(time_min,time_max+time_step,time_step))
                              
                              

Working on Time 6.00Ma

Selected Time is in the stage 6.00Ma to 14.00Ma
Temporary fix for valid time
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_6Ma/m_fig64_11_2_PresentDay_Paleogeog_Matthews2016_6.00Ma.shp']
('', '')
Working on Time 11.00Ma

Selected Time is in the stage 6.00Ma to 14.00Ma
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_6Ma/m_fig64_11_2_PresentDay_Paleogeog_Matthews2016_6.00Ma.shp']
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_14Ma/m_fig62_20_11_PresentDay_Paleogeog_Matthews2016_14.00Ma.shp']
('', '')
Working on Time 16.00Ma

Selected Time is in the stage 14.00Ma to 22.00Ma
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_14Ma/m_fig62_20_11_PresentDay_Paleogeog_Matthews2016_14.00Ma.shp']
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefil

['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_105Ma/m_fig48_117_94_PresentDay_Paleogeog_Matthews2016_105.00Ma.shp']
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_126Ma/m_fig46_135_117_PresentDay_Paleogeog_Matthews2016_126.00Ma.shp']
('', '')
Working on Time 111.00Ma

Selected Time is in the stage 105.00Ma to 126.00Ma
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_105Ma/m_fig48_117_94_PresentDay_Paleogeog_Matthews2016_105.00Ma.shp']
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_126Ma/m_fig46_135_117_PresentDay_Paleogeog_Matthews2016_126.00Ma.shp']
('', '')
Working on Time 116.00Ma

Selected Time is in the stage 105.00Ma to 126.00Ma
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_105Ma/m_fig48_117_94_PresentDay_Pale

['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_218Ma/m_fig36_224_203_PresentDay_Paleogeog_Matthews2016_218.00Ma.shp']
('', '')
Working on Time 206.00Ma

Selected Time is in the stage 195.00Ma to 218.00Ma
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_195Ma/m_fig38_203_179_PresentDay_Paleogeog_Matthews2016_195.00Ma.shp']
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_218Ma/m_fig36_224_203_PresentDay_Paleogeog_Matthews2016_218.00Ma.shp']
('', '')
Working on Time 211.00Ma

Selected Time is in the stage 195.00Ma to 218.00Ma
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_195Ma/m_fig38_203_179_PresentDay_Paleogeog_Matthews2016_195.00Ma.shp']
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_218Ma/m_fig36_224_203_PresentDay_P

['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_302Ma/m_fig26_323_296_PresentDay_Paleogeog_Matthews2016_302.00Ma.shp']
('', '')
Working on Time 301.00Ma

Selected Time is in the stage 287.00Ma to 302.00Ma
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_287Ma/m_fig28_296_285_PresentDay_Paleogeog_Matthews2016_287.00Ma.shp']
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_302Ma/m_fig26_323_296_PresentDay_Paleogeog_Matthews2016_302.00Ma.shp']
('', '')
Working on Time 306.00Ma

Selected Time is in the stage 302.00Ma to 328.00Ma
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_302Ma/m_fig26_323_296_PresentDay_Paleogeog_Matthews2016_302.00Ma.shp']
['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_328Ma/m_fig24_338_323_PresentDay_P

['../paleogeography/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_396Ma/m_fig18_402_380_PresentDay_Paleogeog_Matthews2016_396.00Ma.shp']
('', '')


In [4]:
GW = pygplates.FeatureCollection('GatewayMasks.gpml')

for feature in GW:
    tmp = feature.get(pygplates.PropertyName.create_gpml('subcategory'))  #value('gpml:subcategory')
    print(tmp)


OpenFileForReadingError: Error opening file 'GatewayMasks.gpml' for reading

In [ ]:
print(np.arange(6,401+5,5))